### Coursera - Applied Data Science Capstone - Week 3 assignment

In [1]:
#import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis
#import json # library to handle JSON files
import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
#import matplotlib.cm as cm
#import matplotlib.colors as colors
#from sklearn.cluster import KMeans
from bs4 import BeautifulSoup

#### Input data source from the web page

In [2]:
#input data source
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'lxml')

In [3]:
#grab the table in the web page
table = soup.find('tbody')
print(table.prettify())

<tbody>
 <tr>
  <th>
   Postcode
  </th>
  <th>
   Borough
  </th>
  <th>
   Neighbourhood
  </th>
 </tr>
 <tr>
  <td>
   M1A
  </td>
  <td>
   Not assigned
  </td>
  <td>
   Not assigned
  </td>
 </tr>
 <tr>
  <td>
   M2A
  </td>
  <td>
   Not assigned
  </td>
  <td>
   Not assigned
  </td>
 </tr>
 <tr>
  <td>
   M3A
  </td>
  <td>
   <a href="/wiki/North_York" title="North York">
    North York
   </a>
  </td>
  <td>
   <a href="/wiki/Parkwoods" title="Parkwoods">
    Parkwoods
   </a>
  </td>
 </tr>
 <tr>
  <td>
   M4A
  </td>
  <td>
   <a href="/wiki/North_York" title="North York">
    North York
   </a>
  </td>
  <td>
   <a href="/wiki/Victoria_Village" title="Victoria Village">
    Victoria Village
   </a>
  </td>
 </tr>
 <tr>
  <td>
   M5A
  </td>
  <td>
   <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">
    Downtown Toronto
   </a>
  </td>
  <td>
   <a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">
    Harbourfront
   </a>
  </td>
 </tr>
 <tr>
  

#### Transform the data into a pandas dataframe

In [4]:
#grab the column name of the table
header = table.tr.text.split('\n')
header

['', 'Postcode', 'Borough', 'Neighbourhood', '']

In [5]:
"""
Write a function that get rid of all the empty strings in the list, 
this will be useful in the following operations
"""
def no_emtpy_str(ls):
    ret = []
    for ele in ls:
        if ele != '':
            ret.append(ele)
    return ret

In [6]:
#remove qll empty strings
header = no_emtpy_str(header)
header

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
# instantiate the dataframe
toronto_df = pd.DataFrame(columns=header)
toronto_df.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [8]:
#get the body of the table
body = table.find_all('tr')
print(body)

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

In [9]:
#pop off the column name, which is the first row
body.pop(0)
print(body)

[<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heigh

In [10]:
#Take a look at the structure of a single row
print(body[0].text)
print(body[0].text.split('\n'))


M1A
Not assigned
Not assigned

['', 'M1A', 'Not assigned', 'Not assigned', '']


In [11]:
#insert each row into the datframe
for data in body:
    row = data.text.split('\n')
    if row[2] == 'Not assigned':
        pass
    else:
        if row[3] == 'Not assigned':
            #use row[2] to replace row[3]
            toronto_df = toronto_df.append({'Postcode': row[1],
                              'Borough': row[2],
                              'Neighbourhood': row[2]}, ignore_index=True)
        else:
            toronto_df = toronto_df.append({'Postcode': row[1],
                                          'Borough': row[2],
                                          'Neighbourhood': row[3]}, ignore_index=True)
    
toronto_df.head(10)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [12]:
toronto_df.shape

(211, 3)